In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:

#Create classes for the image processing of the steel defects 
class SteelDefects():
    IMG_SIZE = 200

    Crazing = 'Database/Crazing'
    Inclusions = 'Database/Inclusions'
    Patches = 'Database/Patches'
    PittedSurface = 'Database/PittedSurface'
    RolledInScale = 'Database/RolledInScale'
    Scratches = 'Database/Scratches'
    #We give class values to the images from the different files
    LABELS = {Crazing : 0 , Inclusions : 1 , Patches : 2, PittedSurface : 3, RolledInScale : 4, Scratches : 5}
    #Creating an empty list, training_data, that will be later filled with the images and their labels
    training_data = []
    
    #Setting counters to 0 (to append training samples to training data)
    CrazingCount = 0
    InclusionsCount = 0
    PatchesCount = 0
    PittedSurfaceCount = 0
    RolledInScaleCount = 0
    ScratchesCount = 0
    
    #Creating the data we will train on by associating images to their right class/label
    def associate_training_data(self):
        for label in self.LABELS:
            #print(label)
            for file in tqdm(os.listdir(label)):
                path = os.path.join(label, file) 
                img = cv2.imread(path, cv2.IMREAD_GRAYSCALE) #read images in grey scale 
                #Resizing of the images as defined before: 200*200
                img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                self.training_data.append([np.array(img), np.eye(6)[self.LABELS[label]]])
                #We want to have a one hot vector format with our labels to index images 
                #We used np.eye(6) which is an 6x6 identity matrix (corresponding to our 6 different labels)
            
                
                if label == self.Crazing:
                            self.CrazingCount += 1
                elif label == self.Inclusions:
                            self.InclusionsCount += 1
                elif label == self.Patches:
                            self.PatchesCount += 1
                elif label == self.PittedSurface:
                            self.PittedSurfaceCount += 1
                elif label == self.RolledInScale:
                            self.RolledInScaleCount += 1
                elif label == self.Scratches:
                            self.ScratchesCount += 1
    
        
        #Shuffling the data that will fill the training_data list
        np.random.shuffle(self.training_data)
        np.save("training_data.npy", self.training_data)
        print('Crazings:',self.CrazingCount)
        print('Inclusions:',self.InclusionsCount)
        print('Patches:',self.PatchesCount)
        print('Pitted Surface:',self.PittedSurfaceCount)
        print('Rolled In Scale:',self.RolledInScaleCount)
        print('Scratches:',self.ScratchesCount)

steeldefects = SteelDefects()
steeldefects.associate_training_data()

    

100%|███████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 767.13it/s]


Crazings: 300
Inclusions: 300
Patches: 300
Pitted Surface: 300
Rolled In Scale: 300
Scratches: 300


We observe a perfect balance between classes (300 images per label). It is a necessary condition for our neural network model, which will then not be induced to choose an unbalanced category during its training.

In [3]:
#After setting REBUIlD_DATA to False, we store the training_data as an array in a npy file
training_data = np.load("training_data.npy", allow_pickle = True) 
#Lenght of our training_data
print(len(training_data))

1800


Our sample has 1800 images (300*6 labels)

In [4]:
#printing the pixel matrix and the class of the first image of our sample
print(training_data[0])

[array([[132, 120, 119, ..., 161, 157, 148],
       [126, 128, 126, ..., 178, 177, 163],
       [139, 133, 125, ..., 157, 161, 154],
       ...,
       [142, 144, 170, ..., 154, 159, 148],
       [159, 141, 153, ..., 159, 167, 169],
       [153, 156, 151, ..., 159, 148, 151]], dtype=uint8)
 array([0., 0., 0., 0., 1., 0.])]


The one hot vector is pointing out the class of the first image of the sample.


In [5]:
class Model(nn.Module):
#Building of the model
#We create a class, called Model, that will inherit methods from the nn.Module class
    def __init__(self):
        super().__init__() #The super() function returns an object that represents the parent class
        self.conv1 = nn.Conv2d(1, 32, 3) 
        self.conv2 = nn.Conv2d(32, 64, 3) 
        self.conv3 = nn.Conv2d(64, 128, 3)

        #Creating random data (x), pass it through CNN and reshaping it the right size for passing through the  linear neural network 
        x = torch.randn(200,200).view(-1,1,200,200)
        self._to_linear = None
        self.convs(x)
      
       # Now calling the initialization of the fully connected (fc) neurons network
        self.fc1 = nn.Linear(self._to_linear, 576) 
        self.fc2 = nn.Linear(576, 320) 
        self.fc3 = nn.Linear(320, 6) #output size of 6 as we have 6 differents classes of defects

#Defining the forward method ONLY for the CNN    
    def convs(self, x): 
        x = F.max_pool2d(F.relu(self.conv1(x)), (3, 3)) #pooling with a 3*3 window
        x = F.max_pool2d(F.relu(self.conv2(x)), (3, 3))
        x = F.max_pool2d(F.relu(self.conv3(x)), (3, 3))

        #Making sure that the data passing trough the CNN has the right size before the fc network
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x

#Now defining the entire forward method for the WHOLE model (CNN and linear network)
    def forward(self, x):
        x = self.convs(x) #passing through the convolutional layers
        x = x.view(-1, self._to_linear)  #shaping the data to be flattened
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.softmax(x, dim=1)


model = Model()
print(model)

Model(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=4608, out_features=576, bias=True)
  (fc2): Linear(in_features=576, out_features=320, bias=True)
  (fc3): Linear(in_features=320, out_features=6, bias=True)
)


In [6]:
import torch.optim as optim
#Adam Optimizer for a stochastic optimization (learning rate 1e-3)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.MSELoss()

In [7]:
#Separating xs and ys from our data (xs are pixel values, y is the one hot vector)
X = torch.Tensor([i[0] for i in training_data]).view(-1,200,200)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

In [8]:
Val_percent = 0.3 # we save 30% of our data for testing
val_size = int(len(X)*Val_percent)
print(val_size)

540


Our validation sample size is 540 images. 

In [9]:
train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

print(len(train_X))
print(len(test_X))

1260
540


We are training our model on 1260 images and, as seen before, testing it on 540 pictures.

In [10]:
#batch size = 32 and epochs = 15 for training
BATCH_SIZE = 32
EPOCHS = 15 #number of times we pass data through the model to evaluate it

for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(train_X), BATCH_SIZE)): #From 0, with steps = 32, for the lenght of train_X(=1260)
        batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 200, 200)
        batch_y = train_y[i:i+BATCH_SIZE]

    
        #Setting gradients to 0 before the loss calculation
        optimizer.zero_grad()   
        outputs = model(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()    #Optimizer adjust the weights to lower the loss over time (learning rate of 1e-3)

    print(f"Epoch: {epoch}. Loss: {loss}")
    


  0%|                                                                                           | 0/40 [00:00<?, ?it/s]

Epoch: 0. Loss: 0.1562793254852295


  0%|                                                                                           | 0/40 [00:00<?, ?it/s]

Epoch: 1. Loss: 0.11358287930488586


  0%|                                                                                           | 0/40 [00:00<?, ?it/s]

...

Epoch: 12. Loss: 0.03357209265232086


  0%|                                                                                           | 0/40 [00:00<?, ?it/s]

Epoch: 13. Loss: 0.030377903953194618


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [02:20<00:00,  3.52s/it]

Epoch: 14. Loss: 0.02795458771288395


In [11]:
#Testing the accuracy of the model's prediction
correct = 0
total = 0
#Just evaluating our model accuracy without calculating gradients: using torch.no_grad()
with torch.no_grad(): 
    for i in tqdm(range(len(test_X))):
        real_class = torch.argmax(test_y[i])
        model_out = model(test_X[i].view(-1, 1, 200, 200))[0] 
        predicted_class = torch.argmax(model_out)

        if predicted_class == real_class:
            correct += 1
        total += 1
print("Accuracy: ", round(correct/total*100, 2), "%")

100%|████████████████████████████████████████████████████████████████████████████████| 540/540 [00:48<00:00, 11.22it/s]

Accuracy:  96.3 %


This neural network gives us a 96.3% accuracy in predicting the right classes of the steel defects for the 540 tested pictures.